In [1]:
!pip -q install cadquery trimesh plotly

In [2]:
import cadquery as cq
from cadquery import exporters
from math import sin, cos, pi

# -------------------------
# Wheel parameters (mm)
# -------------------------
outer_d = 600.0        # タイヤ外径（リム外径）
width   = 220.0        # ホイール幅
rim_th  = 25.0         # リム厚み（外周リングの肉厚）
barrel_d = 420.0       # 内側の抜き（リムの内径相当）

hub_d   = 180.0        # ハブ外径
hub_len = 120.0        # ハブの長さ（ホイール幅内）
bore_d  = 70.0         # センターボア径

n_spokes = 6           # スポーク数
spoke_w  = 55.0        # スポーク幅（周方向）
spoke_t  = 35.0        # スポーク厚み（半径方向）
spoke_z  = 70.0        # スポークの厚み（Z方向）

bolt_count = 5
bolt_circle_d = 112.0  # PCD
bolt_hole_d   = 14.0

# -------------------------
# 1) Rim (ring)
# -------------------------
rim = (
    cq.Workplane("XY")
    .circle(outer_d/2.0)
    .circle(barrel_d/2.0)
    .extrude(width)
)

# リム内側を少し“段付き”に
step = (
    cq.Workplane("XY")
    .workplane(offset=width*0.25)
    .circle((barrel_d+30)/2.0)
    .circle((barrel_d-60)/2.0)
    .extrude(width*0.5)
)
rim = rim.union(step)

# -------------------------
# 2) Hub (cylinder) + bore
# -------------------------
hub = cq.Workplane("XY").circle(hub_d/2.0).extrude(hub_len)
hub = hub.translate((0, 0, (width-hub_len)/2.0))  # 中央寄せ

# センターボア（貫通穴）
bore = cq.Workplane("XY").circle(bore_d/2.0).extrude(width)
wheel = rim.union(hub).cut(bore)

# -------------------------
# 3) Spokes (simple rectangular spokes, rotated)
# -------------------------
# スポークが伸びる半径範囲
r_in  = hub_d/2.0 + 15
r_out = (barrel_d/2.0) - 15
spoke_len = r_out - r_in

# 1本のスポーク（中心から右に伸びる形）
spoke = (
    cq.Workplane("XY")
    .center(r_in + spoke_len/2.0, 0)
    .rect(spoke_len, spoke_w)
    .extrude(spoke_z)
)

# Z方向中央寄せ
spoke = spoke.translate((0, 0, (width-spoke_z)/2.0))

# 複数本回転コピー
spokes = cq.Workplane("XY")
for i in range(n_spokes):
    ang = 360.0 * i / n_spokes
    spokes = spokes.union(spoke.rotate((0,0,0), (0,0,1), ang))

wheel = wheel.union(spokes)

# -------------------------
# 4) Bolt holes (PCD)
# -------------------------
bolt_holes = cq.Workplane("XY")
for i in range(bolt_count):
    ang = 2*pi*i/bolt_count
    x = (bolt_circle_d/2.0) * cos(ang)
    y = (bolt_circle_d/2.0) * sin(ang)
    bolt_holes = bolt_holes.union(
        cq.Workplane("XY").center(x, y).circle(bolt_hole_d/2.0).extrude(width)
    )

wheel = wheel.cut(bolt_holes)

# -------------------------
# STL export (higher detail)
# -------------------------
exporters.export(
    wheel,
    "wheel_simple.stl",
    tolerance=0.05,
    angularTolerance=0.3
)
print("Wrote wheel_simple.stl")

Wrote wheel_simple.stl


In [3]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("wheel_simple.stl")

V = m.vertices
edges = m.edges_unique

xs, ys, zs = [], [], []
for a, b in edges:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

mesh_trace = go.Mesh3d(
    x=V[:,0], y=V[:,1], z=V[:,2],
    i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2],
    opacity=0.9
)

edge_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode="lines",
    line=dict(width=1),
    name="edges"
)

fig = go.Figure(data=[mesh_trace, edge_trace])
fig.update_layout(
    scene_aspectmode="data",
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()

In [ ]:
import cadquery as cq
from cadquery import exporters

# ねじパラメータ
major_d = 20.0    # 外径
pitch   = 2.5
length  = 30.0

# ねじ山（ロープ状リッジ）の太さ
ridge_r = 0.8     # ねじ山の太さ

# 芯（円柱）：リッジより内側にする
core_r = major_d/2.0 - 2.2*ridge_r
core_r = max(core_r, 3.0)
core = cq.Workplane("XY").circle(core_r).extrude(length)

# ヘリックス中心線の半径（外径近くを回る）
helix_r = major_d/2.0 - ridge_r
helix = cq.Wire.makeHelix(pitch=pitch, height=length, radius=helix_r)

# ヘリックススタート点は (helix_r, 0, 0)
ridge_profile = cq.Workplane("XY").center(helix_r, 0).circle(ridge_r)

# リッジ
ridge = ridge_profile.sweep(helix, isFrenet=True)

# 合成
screw = core.union(ridge)

exporters.export(
    screw,
    "screw_visible.stl",
    tolerance=0.02,        # 小さいほど細部が出る（重くなる）
    angularTolerance=0.2
)
print("Wrote screw_visible.stl")

In [ ]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("screw_visible.stl")

V = m.vertices
edges = m.edges_unique

xs, ys, zs = [], [], []
for a, b in edges:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

mesh_trace = go.Mesh3d(
    x=V[:,0], y=V[:,1], z=V[:,2],
    i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2],
    opacity=0.9
)

edge_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode="lines",
    line=dict(width=1),
    name="edges"
)

fig = go.Figure(data=[mesh_trace, edge_trace])
fig.update_layout(
    scene_aspectmode="data",
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()